<a href="https://colab.research.google.com/github/Albinator3000/OIDD255_RealEstate_RAG_Model/blob/main/OIDD_255_Real_Estate_Due_Diligence_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Property Due Diligence Assistant - Google Colab Version
### OIDD 2550 - Lab 5: LLM Pitch Project

**This notebook runs in Google Colab using cloud-based LLMs instead of local Ollama**

---

## Setup Instructions

1. **Upload to Google Colab**: File → Upload notebook
2. **Set Runtime**: Runtime → Change runtime type → T4 GPU (free tier)
3. **Run all cells**: Runtime → Run all

---

## Option 1: Use HuggingFace Models (Free, No API Key)

This uses smaller models that run on Colab's free GPU.

In [1]:
# Install dependencies
!pip install -q transformers accelerate langchain langchain-huggingface chromadb \
              pandas numpy matplotlib seaborn plotly folium PyPDF2 python-docx \
              sentence-transformers scikit-learn

print("✅ Dependencies installed!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 3.3 MB/s et

In [3]:
import sys
!{sys.executable} -m pip uninstall -y opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp-proto-http opentelemetry-exporter-otlp-proto-common opentelemetry-proto
!{sys.executable} -m pip install -q opentelemetry-api==1.37.0 opentelemetry-sdk==1.37.0 opentelemetry-exporter-otlp-proto-http==1.37.0 opentelemetry-exporter-otlp-proto-common==1.37.0 opentelemetry-proto==1.37.0

Found existing installation: opentelemetry-api 1.38.0
Uninstalling opentelemetry-api-1.38.0:
  Successfully uninstalled opentelemetry-api-1.38.0
Found existing installation: opentelemetry-sdk 1.38.0
Uninstalling opentelemetry-sdk-1.38.0:
  Successfully uninstalled opentelemetry-sdk-1.38.0
Found existing installation: opentelemetry-exporter-otlp-proto-http 1.37.0
Uninstalling opentelemetry-exporter-otlp-proto-http-1.37.0:
  Successfully uninstalled opentelemetry-exporter-otlp-proto-http-1.37.0
Found existing installation: opentelemetry-exporter-otlp-proto-common 1.38.0
Uninstalling opentelemetry-exporter-otlp-proto-common-1.38.0:
  Successfully uninstalled opentelemetry-exporter-otlp-proto-common-1.38.0
Found existing installation: opentelemetry-proto 1.38.0
Uninstalling opentelemetry-proto-1.38.0:
  Successfully uninstalled opentelemetry-proto-1.38.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 

In [2]:
# Import libraries
import os
import warnings
warnings.filterwarnings('ignore')

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import Document

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from datetime import datetime

print("✅ Libraries imported!")

ModuleNotFoundError: No module named 'langchain_community'

### Load Small LLM Model (Phi-2 or TinyLlama)

We'll use Microsoft's Phi-2 (2.7B parameters) - it's small enough for Colab free tier but still capable.

In [ ]:
# Load Phi-2 model (2.7B parameters - fits in Colab free tier)
print("🔄 Loading Phi-2 model (this takes 2-3 minutes)...")

model_name = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto",  # Automatically uses GPU if available
    load_in_8bit=True   # Quantize to save memory
)

# Create text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.3,
    top_p=0.95,
    repetition_penalty=1.15
)

# Wrap in LangChain
llm = HuggingFacePipeline(pipeline=pipe)

print("✅ Model loaded and ready!")

## Option 2: Use OpenAI API (Requires API Key)

If you have an OpenAI API key, this gives better results than Phi-2.

In [ ]:
# OPTION 2: Use OpenAI (uncomment if you have API key)

# !pip install -q openai langchain-openai

# from langchain_openai import ChatOpenAI
# from google.colab import userdata

# # Store your API key in Colab Secrets (left sidebar → key icon)
# # Name it: OPENAI_API_KEY
# api_key = userdata.get('OPENAI_API_KEY')

# llm = ChatOpenAI(
#     model="gpt-3.5-turbo",
#     temperature=0.3,
#     api_key=api_key
# )

# print("✅ OpenAI LLM ready!")

## Option 3: Use Groq API (Free, Fast)

Groq offers free API access to Llama 3.1 70B - faster than Colab GPU!

In [ ]:
# OPTION 3: Use Groq (free, very fast)

# !pip install -q groq langchain-groq

# from langchain_groq import ChatGroq
# from google.colab import userdata

# # Get free API key at: https://console.groq.com
# # Store in Colab Secrets as: GROQ_API_KEY
# api_key = userdata.get('GROQ_API_KEY')

# llm = ChatGroq(
#     model="llama-3.1-8b-instant",
#     temperature=0.3,
#     api_key=api_key
# )

# print("✅ Groq LLM ready (Llama 3.1 8B)!")

---

## Rest of the Notebook (Same as Local Version)

Everything below this point works identically to the local version.

In [ ]:
# Configuration
CONFIG = {
    'embedding_model': 'sentence-transformers/all-MiniLM-L6-v2',
    'chunk_size': 1000,
    'chunk_overlap': 200,
    'vector_db_path': './chroma_db',
    'top_k': 5
}

RISK_WEIGHTS = {
    'Structural': 0.30,
    'Financial': 0.30,
    'Legal': 0.20,
    'Operational': 0.10,
    'Market': 0.10
}

print("✅ Configuration set")

In [ ]:
# Initialize embeddings
embeddings = HuggingFaceEmbeddings(
    model_name=CONFIG['embedding_model']
)

# Text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CONFIG['chunk_size'],
    chunk_overlap=CONFIG['chunk_overlap']
)

print("✅ Embeddings ready")

In [ ]:
# Sample data (same as local version)
SAMPLE_PROPERTY = {
    'address': '1234 Oak Street, Austin, TX 78701',
    'property_type': 'Multifamily - 4 units',
    'asking_price': 425000,
    'year_built': 1985,
    'sqft': 3200,
}

SAMPLE_INSPECTION = """
PROPERTY INSPECTION REPORT

CRITICAL ISSUES:
1. HVAC System - Unit 2: 18 years old, refrigerant leak. Replacement cost: $5,000-$7,000.
2. Roof: 19 years old, multiple damaged shingles, water staining. Remaining life: 2-3 years. Replacement: $12,000-$15,000.

MODERATE ISSUES:
3. Electrical Panel: Outdated 60 amp service. Upgrade recommended: $3,000-$4,000.
4. Water Heaters: Both over 10 years old. Replacement within 2-3 years: $1,200 each.
5. Foundation: Minor hairline cracks in southeast corner. Monitor for changes.

TOTAL ESTIMATED DEFERRED MAINTENANCE: $25,000-$35,000
"""

DOMAIN_KNOWLEDGE = """
REAL ESTATE VALUATION PRINCIPLES:

Cap Rate Analysis:
- Cap Rate = NOI / Property Value
- Austin multifamily: typically 4.5% - 6.5%

Red Flags:
1. Structural: Foundation cracks > 1/4 inch, mold, roof leaks, HVAC > 15 years
2. Financial: Vacancy > 15%, delinquencies > 30 days, expense ratio > 50%
3. Legal: Unpermitted work, zoning violations
4. Operational: High turnover, month-to-month leases

Renovation Costs (Austin, 2024):
- HVAC: $5,000-$8,000 per unit
- Roof: $8,000-$15,000
- Foundation repair: $2,000-$10,000
"""

print("✅ Sample data created")

In [ ]:
# Create documents
documents = [
    Document(
        page_content=SAMPLE_INSPECTION,
        metadata={'source': 'inspection', 'type': 'inspection'}
    ),
    Document(
        page_content=DOMAIN_KNOWLEDGE,
        metadata={'source': 'knowledge', 'type': 'domain'}
    )
]

# Split and create vector store
split_docs = text_splitter.split_documents(documents)
vectorstore = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings,
    persist_directory=CONFIG['vector_db_path']
)

print(f"✅ Vector database created with {len(split_docs)} chunks")

In [ ]:
# Create RAG chain
PROMPT_TEMPLATE = """
You are a real estate analyst. Use the context below to answer the question.
Be specific and cite numbers from the documents.

Context:
{context}

Question: {question}

Answer:
"""

prompt = PromptTemplate(
    template=PROMPT_TEMPLATE,
    input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": CONFIG['top_k']}),
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

print("✅ RAG chain ready!")

## Test the System

In [ ]:
# Test query
test_query = "What are the critical issues found in the inspection and their costs?"

print(f"🔍 Query: {test_query}\n")
print("🤖 Generating response...\n")

result = qa_chain({"query": test_query})

print("="*80)
print("ANSWER:")
print("="*80)
print(result['result'])
print("\n" + "="*80)

## Risk Assessment

In [ ]:
# Simplified risk scoring for Colab
def calculate_risk_scores():
    """Calculate risk scores based on property data"""

    scores = {
        'Structural': 35,  # Major issues (roof, HVAC)
        'Financial': 55,   # High vacancy
        'Legal': 90,       # Clean
        'Operational': 75, # Manageable
        'Market': 95       # Austin is hot
    }

    # Calculate weighted overall
    overall = sum(scores[cat] * RISK_WEIGHTS[cat] for cat in scores.keys())

    return scores, round(overall, 1)

scores, overall = calculate_risk_scores()

print("📊 RISK ASSESSMENT REPORT")
print("="*80)
for category, score in scores.items():
    weight = RISK_WEIGHTS[category]
    print(f"{category:15} {score:3}/100  (weight: {weight:.0%})")
print("="*80)
print(f"OVERALL SCORE: {overall}/100")
print("="*80)

if overall >= 75:
    decision = "✅ STRONG GO"
elif overall >= 60:
    decision = "⚠️  PROCEED WITH CAUTION"
else:
    decision = "🔴 NO GO"

print(f"\nDECISION: {decision}")

## Visualization

In [ ]:
# Risk score visualization
fig, ax = plt.subplots(figsize=(12, 6))

categories = list(scores.keys())
values = list(scores.values())
colors = ['#ff4444' if v < 60 else '#ffaa00' if v < 80 else '#44ff44' for v in values]

ax.barh(categories, values, color=colors, alpha=0.7, edgecolor='black')
ax.set_xlabel('Risk Score (0-100)', fontsize=12, fontweight='bold')
ax.set_title('Risk Assessment by Category', fontsize=14, fontweight='bold')
ax.set_xlim(0, 100)
ax.axvline(x=60, color='orange', linestyle='--', alpha=0.5)
ax.axvline(x=80, color='green', linestyle='--', alpha=0.5)

for i, (cat, score) in enumerate(zip(categories, values)):
    ax.text(score + 2, i, f'{score:.0f}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("✅ Visualization complete!")

## Valuation & Final Recommendation

In [ ]:
# Valuation calculation
asking_price = 425000
deferred_maintenance = 30000
fair_value = 395000
recommended_offer = fair_value - deferred_maintenance * 0.3

print("💰 VALUATION ANALYSIS")
print("="*80)
print(f"Asking Price:           ${asking_price:,}")
print(f"Fair Market Value:      ${fair_value:,}")
print(f"Deferred Maintenance:   ${deferred_maintenance:,}")
print(f"Recommended Offer:      ${recommended_offer:,.0f} - ${fair_value:,}")
print(f"Discount from Ask:      ${asking_price - recommended_offer:,.0f} ({(asking_price - recommended_offer)/asking_price:.1%})")
print("="*80)

print("\n📋 NEGOTIATION STRATEGY:")
print("1. Open offer at $385,000")
print("2. Request $10,000 seller credit for HVAC replacement")
print("3. Budget $25k-35k for deferred maintenance over next 2 years")
print("4. Walk-away price: $400,000")

## Summary

This Colab version demonstrates the same AI Property Due Diligence system, but using:
- **Cloud GPU** instead of local compute
- **HuggingFace models** (Phi-2) instead of Ollama
- **Same RAG architecture** (ChromaDB + LangChain)

### Advantages of Colab:
- ✅ No local setup required
- ✅ Free GPU access
- ✅ Easy to share with classmates
- ✅ Can use APIs (OpenAI, Groq) for better models

### Disadvantages:
- ❌ Smaller models (Phi-2 2.7B vs Llama 3.1 8B)
- ❌ Session timeout after inactivity
- ❌ Requires API keys for best performance

---

**For your presentation, I recommend using the local version (Ollama + Llama 3.1 8B) for best quality outputs. Use this Colab version as a backup or for sharing with your team!**